In [1]:
import nltk
import utils
import pandas as pd

dream = nltk.corpus.shakespeare.xml("dream.xml")
hamlet = nltk.corpus.shakespeare.xml("hamlet.xml")
macbeth = nltk.corpus.shakespeare.xml("macbeth.xml")
randj = nltk.corpus.shakespeare.xml("r_and_j.xml") 

works = [ 
    {"work_xml": dream,
        "work_name": "Midsummer Night's Dream"}, 
    {"work_xml": hamlet,
        "work_name": "Hamlet"}, 
    {"work_xml": macbeth,
        "work_name": "Macbeth"}, 
    {"work_xml": randj,
        "work_name": "Romeo and Juliet"}
    ]

for title in works:
    main, side = utils.extract_charcs_xml(title["work_xml"], print_charcs=False )
    title["main_charcs"] = main
    title["side_charcs"] = side

    title["parsed_play"] = utils.parse_play_xml(randj)

    title["merged"] = utils.merge_play_data(title["parsed_play"], title["main_charcs"], title["side_charcs"])


all_data = []
for w in works:
    xml_tree = w["work_xml"]
    main, side = utils.extract_charcs_xml(xml_tree, print_charcs=False)
    parsed = utils.parse_play_xml(xml_tree)
    merged = utils.merge_play_data(parsed, main, side)
    stats = utils.summarize_play_stats(merged, main, side)
    df = stats["character_df"]  # extract the actual DataFrame
    all_data.append(df)


# --- Combine and export ---
combined_df = pd.concat(all_data, ignore_index=True)
combined_df.to_csv("all_plays_stats.csv", index=False)





A Midsummer Night's Dream
Acts: 5 | Scenes: 9 | Total Speeches: 500
Main: 21 | Side: 2 | Ratio: 21:2

Act-level speech totals:
  Act 1: 90 speeches
  Act 2: 62 speeches
  Act 3: 180 speeches
  Act 4: 64 speeches
  Act 5: 104 speeches

Top 10 characters by speeches:
                     Play Character  Total Speeches  Scenes Appeared  Speech Share (%)  Avg Speeches/Scene Role Type
A Midsummer Night's Dream    BOTTOM              50                5              10.0               10.00      main
A Midsummer Night's Dream  LYSANDER              50                5              10.0               10.00      main
A Midsummer Night's Dream DEMETRIUS              48                6               9.6                8.00      main
A Midsummer Night's Dream   THESEUS              48                3               9.6               16.00      main
A Midsummer Night's Dream    HERMIA              47                4               9.4               11.75      main
A Midsummer Night's Dream    QU